In [ ]:
import numpy as np

from tensorflow.keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, Input
from tensorflow.keras.models import Model, Sequential

from tensorflow.keras.datasets import mnist
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow_datasets as tfds
import tensorflow as tf

In [ ]:
!wget http://vision.stanford.edu/aditya86/ImageNetDogs/images.tar
!tar -xf images.tar

import os
import random
import shutil

source_dir = "Images"
test_dir = "test_Images"

if not os.path.exists(test_dir):
    os.makedirs(test_dir)

for subdir in os.listdir(source_dir):
    subdir_path = os.path.join(source_dir, subdir)

    if os.path.isdir(subdir_path):
        test_subdir_path = os.path.join(test_dir, subdir)
        if not os.path.exists(test_subdir_path):
            os.makedirs(test_subdir_path)
        images = [f for f in os.listdir(subdir_path) if os.path.isfile(os.path.join(subdir_path, f))]

        num_images_to_move = min(10, len(images)) # Ensure we don't try to move more images than exist
        selected_images = random.sample(images, num_images_to_move)
        for image_file in selected_images:
            source_image_path = os.path.join(subdir_path, image_file)
            test_image_path = os.path.join(test_subdir_path, image_file)
            shutil.move(source_image_path, test_image_path)

One of the easiest ways to prepare image data for training is by using data generators. They eliminate the need to manually load data into memory, helping prevent memory overflow issues. Additionally, data augmentation and preprocessing can be conveniently performed during this stage.

In [ ]:
datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.3,
    shear_range=0.01,
    zoom_range=0.2,
    rotation_range=1,
    horizontal_flip=True
)

train_generator = datagen.flow_from_directory(
    source_dir,
    target_size=(192, 192),
    class_mode="categorical",
    batch_size=32,
    shuffle=True,
    subset='training',
)

val_generator = datagen.flow_from_directory(
    source_dir,
    target_size=(192, 192),
    class_mode="categorical",
    batch_size=32,
    subset='validation'
)

testgen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
)

test_generator = testgen.flow_from_directory(
    test_dir,
    target_size=(192, 192),
    class_mode="categorical",
    batch_size=32,
)

## Task
Try to predict breed of a dog from an image using cnn. You can directly use train_generator, val_generator in a fit function

As you probably noticed it's not so easy. We can enhance the process by using a CNN pretrained on a set of general images. Of course, it cannot be used directly, but the whole feature extraction part can be copied and then maybe slightly adjusted.

In [ ]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(192, 192, 3))
for layer in base_model.layers:
  layer.trainable = False
base_model.summary()

## Task

The model is quite big, but it's already trained so we froze the weights to not change them during the training phase. After the training is finished the weights can be unfrozen and the training can be repeated to adjust them even better. Now we have only the convolutional part, add the missing part to perform a classification, and compare the results with a model built from scratch.

Image segmentation is an image-to-image task where the output is a binary image with the same shape as the input showing the location of a given object e.g. roads, humans, or signs.

A U-net architecture is a popular model used in this task. It allows us to capture not only local patterns and map them to the output. It's presented in a picture below. Of course, it's just an example and the number of layers or number of neurons do not have to be copied one to one.

<img src="https://media.geeksforgeeks.org/wp-content/uploads/20220614121231/Group14.jpg">

https://www.geeksforgeeks.org/u-net-architecture-explained/

## Task
Download a retina blood vessel dataset https://www.kaggle.com/datasets/abdallahwagih/retina-blood-vessel create and train an U-net. You can use transfer learning, but it's not a must